In [5]:
# pip install mlxtend pandas

In [7]:
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules

# Carrega a base
base = pd.read_csv('../Bases/Criados/6.1_base_final.csv')

# Mapeia a variável alvo
base['Doença_cardíaca'] = base['Doença_cardíaca'].map({1: 'Doenca', 2: 'Saudavel'})

# Converte tudo para string
df = base.astype(str)

# One-hot encoding
df_encoded = pd.get_dummies(df)

# Aplica Apriori com suporte mais alto para reduzir o número de combinações
frequent_items = apriori(df_encoded, min_support=0.2, use_colnames=True, low_memory=True)

# Gera regras com confiança alta
regras = association_rules(frequent_items, metric="confidence", min_threshold=0.95)

# Filtra para regras com consequente sendo a classe alvo
regras_alvo = regras[
    (regras['consequents'].astype(str).str.contains("Doença_cardíaca_Doenca")) |
    (regras['consequents'].astype(str).str.contains("Doença_cardíaca_Saudavel"))
]

# Ordena as regras por lift decrescente
regras_ordenadas = regras_alvo.sort_values(by='lift', ascending=False).reset_index(drop=True)

# Converte conjuntos para strings legíveis
regras_ordenadas['antecedents'] = regras_ordenadas['antecedents'].apply(lambda x: ', '.join(list(x)))
regras_ordenadas['consequents'] = regras_ordenadas['consequents'].apply(lambda x: ', '.join(list(x)))

# Relatório final completo (sem arredondar nada)
relatorio_final = regras_ordenadas[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'leverage', 'conviction']].head(10)

# Exibe relatório
print("📋 RELATÓRIO COMPLETO DAS REGRAS DE ASSOCIAÇÃO")
print("="*60)
print(relatorio_final.to_string(index=False))

📋 RELATÓRIO COMPLETO DAS REGRAS DE ASSOCIAÇÃO
                                                                                                                                       antecedents                                                                                                consequents  support  confidence     lift  leverage  conviction
                                                       Frequência_falta_de_interesse_1.0, Quantidade_de_trabalhos_0, Diabetes_2.0, Doença_mental_2                            Jornada_de_Trabalho_0, Uso_aparelhos_com_nicotina_4.0, Doença_cardíaca_Saudavel 0.209961    0.954324 2.474410  0.125108   13.449491
                                                           Diabetes_2.0, Frequência_falta_de_interesse_1.0, Jornada_de_Trabalho_0, Doença_mental_2                        Doença_cardíaca_Saudavel, Quantidade_de_trabalhos_0, Uso_aparelhos_com_nicotina_4.0 0.209961    0.954324 2.474410  0.125108   13.449491
                                    

In [8]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Seu código para carregar e preparar a base, encoding, Apriori etc, vai aqui
base = pd.read_csv('../Bases/Criados/6.1_base_final.csv')
base['Doença_cardíaca'] = base['Doença_cardíaca'].map({1: 'Doenca', 2: 'Saudavel'})
df = base.astype(str)
df_encoded = pd.get_dummies(df)
frequent_items = apriori(df_encoded, min_support=0.2, use_colnames=True, low_memory=True)
regras = association_rules(frequent_items, metric="confidence", min_threshold=0.95)
regras_alvo = regras[
    (regras['consequents'].astype(str).str.contains("Doença_cardíaca_Doenca")) |
    (regras['consequents'].astype(str).str.contains("Doença_cardíaca_Saudavel"))
]
regras_ordenadas = regras_alvo.sort_values(by='lift', ascending=False)

# Função para interpretar e gerar o relatório
def gerar_relatorio_amigavel(df_regras):
    relatorio = "📋 RELATÓRIO COMPLETO DAS REGRAS DE ASSOCIAÇÃO\n"
    relatorio += "="*60 + "\n\n"
    
    for i, row in df_regras.iterrows():
        antecedents = ", ".join(list(row['antecedents']))
        consequents = ", ".join(list(row['consequents']))
        support = row['support']
        confidence = row['confidence']
        lift = row['lift']
        
        relatorio += f"Regra {i+1}:\n"
        relatorio += f"Se uma pessoa tem [{antecedents}], então provavelmente tem [{consequents}].\n"
        relatorio += f"Suporte da regra: {support:.6f} (proporção dos dados onde isso ocorre)\n"
        relatorio += f"Confiança da regra: {confidence:.6f} (probabilidade da consequência dado o antecedente)\n"
        relatorio += f"Lift da regra: {lift:.6f} (força da associação, >1 é significativo)\n"
        relatorio += "-"*60 + "\n"
    
    return relatorio

print(gerar_relatorio_amigavel(regras_ordenadas.head(5)))  # mostra as 5 principais regras


📋 RELATÓRIO COMPLETO DAS REGRAS DE ASSOCIAÇÃO

Regra 113604:
Se uma pessoa tem [Frequência_falta_de_interesse_1.0, Quantidade_de_trabalhos_0, Diabetes_2.0, Doença_mental_2], então provavelmente tem [Jornada_de_Trabalho_0, Uso_aparelhos_com_nicotina_4.0, Doença_cardíaca_Saudavel].
Suporte da regra: 0.209961 (proporção dos dados onde isso ocorre)
Confiança da regra: 0.954324 (probabilidade da consequência dado o antecedente)
Lift da regra: 2.474410 (força da associação, >1 é significativo)
------------------------------------------------------------
Regra 113603:
Se uma pessoa tem [Diabetes_2.0, Frequência_falta_de_interesse_1.0, Jornada_de_Trabalho_0, Doença_mental_2], então provavelmente tem [Doença_cardíaca_Saudavel, Quantidade_de_trabalhos_0, Uso_aparelhos_com_nicotina_4.0].
Suporte da regra: 0.209961 (proporção dos dados onde isso ocorre)
Confiança da regra: 0.954324 (probabilidade da consequência dado o antecedente)
Lift da regra: 2.474410 (força da associação, >1 é significativo)
